Experiment  
i tried to do undersampling.  
hyper parameters were tuned by optuna for undersampled data.  
  
public score: 1.36304  
  
predicted probabilities for each class are almost the same (about 0.25) <- df_sub  
i suspect that it is difficult to predict class.

【実験】  
アンダーサンプリングしたデータで学習・予測してみた。  
ハイパーパラメータはoptunaでチューニングした。  
  
public score: 1.3604  
  
予測された確率はどのクラスもほぼ同じ（だいたい0.25付近）<- df_sub参照  
クラスを予測するのは難しいのではないかと思う。  
<- Xからyを予測するとして、yの予測にXが全然寄与してない。Xを使っても全然予測できない。予測できていない。

In [ ]:
import os
import numpy as np
import pandas as pd

SEED = 1380

In [ ]:
df_train = pd.read_csv("../input/tabular-playground-series-may-2021/train.csv")

data1 = df_train[df_train["target"] == "Class_1"]
data2 = df_train[df_train["target"] == "Class_2"]
data3 = df_train[df_train["target"] == "Class_3"]
data4 = df_train[df_train["target"] == "Class_4"]

print("Class_1:", len(data1))
print("Class_2:", len(data2))
print("Class_3:", len(data3))
print("Class_4:", len(data4))

In [ ]:
data1 = data1
data2 = data2.sample(n=len(data1), axis=0, random_state=SEED)
data3 = data3.sample(n=len(data1), axis=0, random_state=SEED)
data4 = data4.sample(n=len(data1), axis=0, random_state=SEED)

print("Class_1:", len(data1))
print("Class_2:", len(data2))
print("Class_3:", len(data3))
print("Class_4:", len(data4))

dataset1 = pd.concat([data1, data2, data3, data4], axis=0).reset_index(drop=True)
print("dataset1 shape:", dataset1.shape)

In [ ]:
X = dataset1.drop(["id", "target"], axis=1)
y = dataset1.target

ctoi = {"Class_1": 0, "Class_2": 1, "Class_3": 2, "Class_4": 3}
y.replace(ctoi, inplace=True)

X = np.array(X)
y = np.array(y)

print(X.shape, y.shape)

In [ ]:
df_test = pd.read_csv("../input/tabular-playground-series-may-2021/test.csv")
X_test = df_test.drop(["id"], axis=1)
X_test = np.array(X_test)

print(X_test.shape)

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
import xgboost as xgb

In [ ]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)

In [ ]:
params = {
    "tree_method": "gpu_hist",
    "objective": "multi:softprob",  # predict probability / 確率を出力
    "num_class": 4,
    "eval_metric": "mlogloss",
    "eta": 0.006,
    "max_depth": 4,
    "gamma": 0.8481259071157187,
    "min_child_weight": 0.5702096116340974,
    "colsample_bytree": 0.2235888371622891,
    "subsample": 0.6269307669980926,
    "lambda": 0.05686076044791255,
    "alpha": 9.971025438987308,
    "max_bin": 900,
    "seed": SEED
}

In [ ]:
preds = np.zeros((50000, 4))
d_test = xgb.DMatrix(X_test)

for tr_id, vl_id in kf.split(X, y):
    print("#"*70)
    
    X_train, X_valid = X[tr_id, :], X[vl_id, :]
    y_train, y_valid = y[tr_id], y[vl_id]

    d_train = xgb.DMatrix(X_train, y_train)
    d_valid = xgb.DMatrix(X_valid, y_valid)

    model = xgb.train(params=params,
                      dtrain=d_train,
                      num_boost_round=100000,
                      early_stopping_rounds=300,
                      evals=[(d_train, "train"), (d_valid, "valid")],
                      verbose_eval=1000)
    pred = model.predict(d_test, ntree_limit=model.best_ntree_limit)
    preds += pred
    
preds = preds / 5

In [ ]:
df_sub = pd.read_csv("../input/tabular-playground-series-may-2021/sample_submission.csv")
df_sub[["Class_1", "Class_2", "Class_3", "Class_4"]] = preds

df_sub.head()

In [ ]:
df_sub

In [ ]:
df_sub.to_csv("XGBoost_undersampling.csv", index=False)